In [3]:
# First run this cell
!pip install terminaltables
from datetime import datetime
import argparse
import random
import pickle
import codecs
import json
import os
import nltk
import torch
import numpy as np
from pprint import pprint
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from terminaltables import AsciiTable

import pandas as pd


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\Users\KTH\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Preprocess data

In [4]:
dataset_file = './data/reviews.csv'

In [5]:
data = pd.read_csv("./data/Reviews.csv")

In [6]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
data = data.dropna()
data = data.reset_index(drop=True)
data = data.loc[:, ['Summary', 'Text']]
data.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
len(data)

568411

In [9]:
source_w2i = {}
source_i2w = []
target_w2i = {}
target_i2w = []

# The padding symbol will be used to ensure that all tensors in a batch
# have equal length.
PADDING_SYMBOL = ' '
source_w2i[PADDING_SYMBOL] = 0
source_i2w.append( PADDING_SYMBOL )
target_w2i[PADDING_SYMBOL] = 0
target_i2w.append( PADDING_SYMBOL )

START_SYMBOL = '<START>'
END_SYMBOL = '<END>'
UNK_SYMBOL = '<UNK>'
source_w2i[START_SYMBOL] = 1
source_i2w.append( START_SYMBOL )
target_w2i[START_SYMBOL] = 1
target_i2w.append( START_SYMBOL )
source_w2i[END_SYMBOL] = 2
source_i2w.append( END_SYMBOL )
target_w2i[END_SYMBOL] = 2
target_i2w.append( END_SYMBOL )
source_w2i[UNK_SYMBOL] = 3
source_i2w.append( UNK_SYMBOL )
target_w2i[UNK_SYMBOL] = 3
target_i2w.append( UNK_SYMBOL )

# Max number of words to be predicted if <END> symbol is not reached
MAX_PREDICTIONS = 20

In [10]:
class AmazonDataset(Dataset) :
    """
    A dataset with source sentences and their respective translations
    into the target language.

    Each sentence is represented as a list of word IDs. 
    """
    def __init__( self, data, record_symbols=True ) :
        try :
            nltk.word_tokenize("hi there.")
        except LookupError:
            nltk.download('punkt')
        self.source_list = []
        self.target_list = []
        # Read the datafile
        
        for i in range(len(data)):
            s = data.Text[i]
            t = data.Summary[i]
            source_sentence = []
            for w in nltk.word_tokenize(s) :
                w = w.lower()
                if w not in source_i2w and record_symbols :
                    source_w2i[w] = len(source_i2w)
                    source_i2w.append( w )
                source_sentence.append( source_w2i.get(w, source_w2i[UNK_SYMBOL]) )
            source_sentence.append(source_w2i[END_SYMBOL])
            self.source_list.append( source_sentence )
            target_sentence = []
            for w in nltk.word_tokenize(t) :
                w = w.lower()
                if w not in target_i2w and record_symbols :
                    target_w2i[w] = len(target_i2w)
                    target_i2w.append( w )
                target_sentence.append( target_w2i.get(w, target_w2i[UNK_SYMBOL]) )
            target_sentence.append(target_w2i[END_SYMBOL])
            self.target_list.append( target_sentence )

    def __len__(self) :
        return len(self.source_list)

    def __getitem__(self, idx) :
        return self.source_list[idx], self.target_list[idx]

In [11]:
# dataset = AmazonDataset(data)

In [12]:
# torch.save(dataset, 'amazon_dataset_py')

In [13]:
loaded_dataset = torch.load('./data/amazon_dataset_py')

In [14]:
len(loaded_dataset)

568411

In [15]:
def pad_sequence(batch, pad_source=source_w2i[PADDING_SYMBOL], pad_target=target_w2i[PADDING_SYMBOL]):
    source, target = zip(*batch)
    max_source_len = max(map(len, source))
    max_target_len = max(map(len, target))
    padded_source = [[b[i] if i < len(b) else pad_source for i in range(max_source_len)] for b in source]
    padded_target = [[l[i] if i < len(l) else pad_target for i in range(max_target_len)] for l in target]
    return padded_source, padded_target